In [ ]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [1]:
# import GYM stuff
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

# import helpers
import numpy as np
import random
import os

# Import stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [6]:
Discrete(3).sample()

0

In [7]:
Box(0,1,shape=(3,3)).sample()

array([[0.354973  , 0.54511017, 0.12367824],
       [0.7971712 , 0.5502458 , 0.09646376],
       [0.42969355, 0.5473869 , 0.2237898 ]], dtype=float32)

In [8]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 87, 239,  50],
       [232,  91, 239],
       [208, 234,  30]])

In [9]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([77.76645], dtype=float32))

In [10]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([63.62854], dtype=float32))])

In [11]:
MultiBinary(4).sample()

array([1, 0, 1, 0], dtype=int8)

In [12]:
MultiDiscrete([5,2,2]).sample()

array([4, 1, 1], dtype=int64)

# 3. Building an Environment

In [13]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [14]:
env=ShowerEnv()

C:\Users\witeo\anaconda3\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [23]:
env.observation_space.sample()

array([91.91801], dtype=float32)

In [24]:
env.reset()

array([36.])

In [25]:
from stable_baselines3.common.env_checker import check_env

In [26]:
check_env(env, warn=True)

AssertionError: The observation returned by the `reset()` method does not match the given observation space

# 4. Test Environment

In [27]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-10
Episode:2 Score:-22
Episode:3 Score:-2
Episode:4 Score:20
Episode:5 Score:-36


In [ ]:
env.close()

# 5. Train Model

In [28]:
log_path = os.path.join('Training', 'Logs')

In [29]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [30]:
model.learn(total_timesteps=400000)

Logging to Training\Logs\PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -26.4    |
| time/              |          |
|    fps             | 293      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | -31.2      |
| time/                   |            |
|    fps                  | 247        |
|    iterations           | 2          |
|    time_elapsed         | 16         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01219219 |
|    clip_fraction        | 0.0565     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.09      |
|    explained_variance   | -0.000323  |
|    learning_

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -21          |
| time/                   |              |
|    fps                  | 218          |
|    iterations           | 11           |
|    time_elapsed         | 103          |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0043389853 |
|    clip_fraction        | 0.0628       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.714       |
|    explained_variance   | 0.000613     |
|    learning_rate        | 0.0003       |
|    loss                 | 45           |
|    n_updates            | 100          |
|    policy_gradient_loss | -0.0045      |
|    value_loss           | 90.8         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -13.5       |
| time/                   |             |
|    fps                  | 237         |
|    iterations           | 21          |
|    time_elapsed         | 180         |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.004574965 |
|    clip_fraction        | 0.0295      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.465      |
|    explained_variance   | 0.000519    |
|    learning_rate        | 0.0003      |
|    loss                 | 48.5        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.00178    |
|    value_loss           | 109         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -4.98        |
| time/                   |              |
|    fps                  | 231          |
|    iterations           | 31           |
|    time_elapsed         | 273          |
|    total_timesteps      | 63488        |
| train/                  |              |
|    approx_kl            | 0.0014371906 |
|    clip_fraction        | 0.0175       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.346       |
|    explained_variance   | 0.015        |
|    learning_rate        | 0.0003       |
|    loss                 | 57.4         |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.000919    |
|    value_loss           | 114          |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -7.16        |
| time/                   |              |
|    fps                  | 227          |
|    iterations           | 41           |
|    time_elapsed         | 369          |
|    total_timesteps      | 83968        |
| train/                  |              |
|    approx_kl            | 0.0006410368 |
|    clip_fraction        | 0.0216       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.191       |
|    explained_variance   | 0.109        |
|    learning_rate        | 0.0003       |
|    loss                 | 64.7         |
|    n_updates            | 400          |
|    policy_gradient_loss | -0.00268     |
|    value_loss           | 132          |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60            |
|    ep_rew_mean          | -7.56         |
| time/                   |               |
|    fps                  | 267           |
|    iterations           | 154           |
|    time_elapsed         | 1177          |
|    total_timesteps      | 315392        |
| train/                  |               |
|    approx_kl            | 9.0865884e-05 |
|    clip_fraction        | 0.00244       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0417       |
|    explained_variance   | 0.845         |
|    learning_rate        | 0.0003        |
|    loss                 | 42.7          |
|    n_updates            | 1530          |
|    policy_gradient_loss | -0.000536     |
|    value_loss           | 132           |
-------------------------------------------
-------------------------------------------
| rollout/                |     

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60            |
|    ep_rew_mean          | -2.08         |
| time/                   |               |
|    fps                  | 267           |
|    iterations           | 163           |
|    time_elapsed         | 1247          |
|    total_timesteps      | 333824        |
| train/                  |               |
|    approx_kl            | 0.00011087078 |
|    clip_fraction        | 0.00371       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0761       |
|    explained_variance   | 0.848         |
|    learning_rate        | 0.0003        |
|    loss                 | 57.6          |
|    n_updates            | 1620          |
|    policy_gradient_loss | -0.000651     |
|    value_loss           | 125           |
-------------------------------------------
------------------------------------------
| rollout/                |      

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60            |
|    ep_rew_mean          | -19.6         |
| time/                   |               |
|    fps                  | 267           |
|    iterations           | 172           |
|    time_elapsed         | 1315          |
|    total_timesteps      | 352256        |
| train/                  |               |
|    approx_kl            | 0.00012752684 |
|    clip_fraction        | 0.00801       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0708       |
|    explained_variance   | 0.812         |
|    learning_rate        | 0.0003        |
|    loss                 | 64.1          |
|    n_updates            | 1710          |
|    policy_gradient_loss | -0.0005       |
|    value_loss           | 138           |
-------------------------------------------
------------------------------------------
| rollout/                |      

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60            |
|    ep_rew_mean          | -2.66         |
| time/                   |               |
|    fps                  | 267           |
|    iterations           | 181           |
|    time_elapsed         | 1384          |
|    total_timesteps      | 370688        |
| train/                  |               |
|    approx_kl            | 0.00048509613 |
|    clip_fraction        | 0.0063        |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0448       |
|    explained_variance   | 0.824         |
|    learning_rate        | 0.0003        |
|    loss                 | 76            |
|    n_updates            | 1800          |
|    policy_gradient_loss | 0.00166       |
|    value_loss           | 141           |
-------------------------------------------
-------------------------------------------
| rollout/                |     

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -4.08        |
| time/                   |              |
|    fps                  | 269          |
|    iterations           | 190          |
|    time_elapsed         | 1446         |
|    total_timesteps      | 389120       |
| train/                  |              |
|    approx_kl            | 9.327184e-05 |
|    clip_fraction        | 0.00264      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.0353      |
|    explained_variance   | 0.847        |
|    learning_rate        | 0.0003       |
|    loss                 | 61           |
|    n_updates            | 1890         |
|    policy_gradient_loss | -0.000386    |
|    value_loss           | 134          |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

# 6. Save Model

In [31]:
model.save('PPO')

In [33]:
evaluate_policy(model, env, n_eval_episodes=10)

(-36.0, 48.0)